In [2]:
import pandas as pd
import sqlite3

In [5]:
CFG = {
    "DB_FILENAME": "ui/zizi.sqlite",
    "DEBUG_FLAG": True,
    "SQL_EXECUTION_FLAG": False,
}

In [6]:
def debug_print(msg, debug=CFG["DEBUG_FLAG"]):
    if debug and msg:
        # st.write(f"[DEBUG] {str(msg)}")
        print(f"[DEBUG] {str(msg)}")

#############################
#  DB related  (2nd)
#############################
class DBConn(object):
    def __init__(self, db_file=CFG["DB_FILENAME"]):
        self.conn = sqlite3.connect(db_file)

    def __enter__(self):
        return self.conn

    def __exit__(self, type, value, traceback):
        self.conn.close()


def db_run_sql(sql_stmt, conn=None, debug=CFG["DEBUG_FLAG"]):
    """handles both select and insert/update/delete
    """
    if not sql_stmt or conn is None:
        return None
    
    debug_print(sql_stmt, debug=debug)

    if sql_stmt.lower().strip().startswith("select"):
        return pd.read_sql(sql_stmt, conn)
    
    cur = conn.cursor()
    cur.executescript(sql_stmt)
    conn.commit()
    # conn.close()
    return None


def db_execute(sql_statement, 
               debug=CFG["DEBUG_FLAG"], 
               execute_flag=CFG["SQL_EXECUTION_FLAG"],):
    """handles insert/update/delete
    """
    with DBConn() as _conn:
        debug_print(sql_statement, debug=debug)
        if execute_flag:
            _conn.execute(sql_statement)
            _conn.commit()
        else:
            print("[WARN] SQL Execution is off ! ")   

In [7]:
file_csv = "t_zi_part_cleanup-revised.csv"

In [10]:
df = pd.read_csv(file_csv)[["zi", "u_id"]]

In [11]:
df

,zi,u_id
0,奉,3622
1,平,8141
2,丘,8631
3,尺,2327
4,印,12054
...,...,...
327,皿,9
328,生,9303
329,手,9434
330,隧,9757


In [17]:
with DBConn() as _conn:
    sql_stmt = """
        --select * from t_zi_part limit 10
        select count(*) from t_zi_part where is_active='Y'
    """
    df_ = pd.read_sql(sql_stmt, _conn).fillna("")

In [18]:
df_ 

,count(*)
0,6543


In [16]:
with DBConn() as _conn:
    df.to_sql("w_zi_part_cleanup", _conn, index=False, if_exists="replace")

In [19]:
with DBConn() as _conn:
    sql_stmt = """
        --select * from t_zi_part limit 10
        select count(*) from w_zi_part_cleanup 
    """
    df_ = pd.read_sql(sql_stmt, _conn).fillna("")

In [20]:
df_

,count(*)
0,332


In [34]:
with DBConn() as _conn:
    sql_stmt = """
        --select * from t_zi_part limit 10
        select count(*) from w_zi_part_cleanup w where exists (
            select 1 from t_zi_part t where t.is_active='Y' and t.u_id = w.u_id
        )
    """
    
    sql_stmt = """
    with z as (
        select * from t_zi_part where 1=1 
            and (zi_left_up is null or trim(zi_left_up) = '')
            and (zi_left is null or trim(zi_left) = '')
            and (zi_left_down is null or trim(zi_left_down) = '')
            and (zi_right_up is null or trim(zi_right_up) = '')
            and (zi_right is null or trim(zi_right) = '')
            and (zi_right_down is null or trim(zi_right_down) = '')
            and (zi_up is null or trim(zi_up) = '')
            and (zi_down is null or trim(zi_down) = '')
            and (zi_mid is null or trim(zi_mid) = '')
            and (zi_mid_in is null or trim(zi_mid_in) = '')
            and (zi_mid_out is null or trim(zi_mid_out) = '')
            and is_active = 'Y'        
    )
    select z.* from z join w_zi_part_cleanup w on z.zi = w.zi and z.u_id = w.u_id
    """
    sql_stmt = """
    with z as (
        select * from t_zi_part where 1=1 
            and (zi_left_up is null or trim(zi_left_up) = '')
            and (zi_left is null or trim(zi_left) = '')
            and (zi_left_down is null or trim(zi_left_down) = '')
            and (zi_right_up is null or trim(zi_right_up) = '')
            and (zi_right is null or trim(zi_right) = '')
            and (zi_right_down is null or trim(zi_right_down) = '')
            and (zi_up is null or trim(zi_up) = '')
            and (zi_down is null or trim(zi_down) = '')
            and (zi_mid is null or trim(zi_mid) = '')
            and (zi_mid_in is null or trim(zi_mid_in) = '')
            and (zi_mid_out is null or trim(zi_mid_out) = '')
            and is_active =  'Y'        
    )
    update t_zi_part set is_active = '' where u_id in (
        select u_id from z
    )
    --select w.* from  w_zi_part_cleanup w  join z on z.zi = w.zi and z.u_id = w.u_id
    """
    # df_2 = pd.read_sql(sql_stmt, _conn).fillna("")
    db_run_sql(sql_stmt, conn=_conn)

[DEBUG] 
    with z as (
        select * from t_zi_part where 1=1 
            and (zi_left_up is null or trim(zi_left_up) = '')
            and (zi_left is null or trim(zi_left) = '')
            and (zi_left_down is null or trim(zi_left_down) = '')
            and (zi_right_up is null or trim(zi_right_up) = '')
            and (zi_right is null or trim(zi_right) = '')
            and (zi_right_down is null or trim(zi_right_down) = '')
            and (zi_up is null or trim(zi_up) = '')
            and (zi_down is null or trim(zi_down) = '')
            and (zi_mid is null or trim(zi_mid) = '')
            and (zi_mid_in is null or trim(zi_mid_in) = '')
            and (zi_mid_out is null or trim(zi_mid_out) = '')
            and is_active =  'Y'        
    )
    update t_zi_part set is_active = '' where u_id in (
        select u_id from z
    )
    --select w.* from  w_zi_part_cleanup w  join z on z.zi = w.zi and z.u_id = w.u_id
    


In [35]:
with DBConn() as _conn:
    sql_stmt = """
        --select * from t_zi_part limit 10
        select count(*) from w_zi_part_cleanup w where exists (
            select 1 from t_zi_part t where t.is_active='Y' and t.u_id = w.u_id
        )
    """
    
    sql_stmt = """
    with z as (
        select * from t_zi_part where 1=1 
            and (zi_left_up is null or trim(zi_left_up) = '')
            and (zi_left is null or trim(zi_left) = '')
            and (zi_left_down is null or trim(zi_left_down) = '')
            and (zi_right_up is null or trim(zi_right_up) = '')
            and (zi_right is null or trim(zi_right) = '')
            and (zi_right_down is null or trim(zi_right_down) = '')
            and (zi_up is null or trim(zi_up) = '')
            and (zi_down is null or trim(zi_down) = '')
            and (zi_mid is null or trim(zi_mid) = '')
            and (zi_mid_in is null or trim(zi_mid_in) = '')
            and (zi_mid_out is null or trim(zi_mid_out) = '')
            and is_active = 'Y'        
    )
    select z.* from z join w_zi_part_cleanup w on z.zi = w.zi and z.u_id = w.u_id
    """
    sql_stmt = """
    with z as (
        select * from t_zi_part where 1=1 
            and (zi_left_up is null or trim(zi_left_up) = '')
            and (zi_left is null or trim(zi_left) = '')
            and (zi_left_down is null or trim(zi_left_down) = '')
            and (zi_right_up is null or trim(zi_right_up) = '')
            and (zi_right is null or trim(zi_right) = '')
            and (zi_right_down is null or trim(zi_right_down) = '')
            and (zi_up is null or trim(zi_up) = '')
            and (zi_down is null or trim(zi_down) = '')
            and (zi_mid is null or trim(zi_mid) = '')
            and (zi_mid_in is null or trim(zi_mid_in) = '')
            and (zi_mid_out is null or trim(zi_mid_out) = '')
            and is_active =  'Y'        
    )
    select w.* from  w_zi_part_cleanup w  join z on z.zi = w.zi and z.u_id = w.u_id
    """
    df_2 = pd.read_sql(sql_stmt, _conn).fillna("")


In [36]:
df_2.shape

(0, 2)

In [28]:
df_2.head(2)

,zi,u_id,zi_left_up,zi_left,zi_left_down,zi_up,zi_mid,zi_down,zi_right_up,zi_right,zi_right_down,zi_mid_out,zi_mid_in,ts,desc_cn,is_active,id_shuowen,hsk_note,desc_en
0,奉,3622,,,,,,,,,,,,2024-01-18 02:09:12,承也。从手从廾，𡴀聲。,Y,1749,in HSK,
1,平,8141,,,,,,,,,,,,2024-01-15 12:08:23,語平舒也。从亏从八。八，分也。爰禮說。,Y,3056,in HSK,


## inactivate duplicates

In [37]:
with DBConn() as _conn:
    sql_stmt = """
        with dup_zi as (
            select zi,count(*) from t_zi_part where is_active='Y' and cast(u_id as int) > 0 group by zi having count(*) > 1
        )
        , zi_2 as (
            select zi,min(cast(u_id as int)) as u_id
            from t_zi_part 
            where is_active='Y' and cast(u_id as int) > 0 and zi in (select zi from dup_zi)
            group by zi 
        )
        select * from t_zi_part where zi in (select zi from dup_zi) order by zi;
    """
    df_dup = pd.read_sql(sql_stmt, _conn).fillna("")


In [39]:
df_dup.to_csv("t_zi_part_dup-20240204.csv", index=False)

In [40]:
with DBConn() as _conn:
    sql_stmt = """
        with dup_zi as (
            select zi,count(*) from t_zi_part where is_active='Y' and cast(u_id as int) > 0 group by zi having count(*) > 1
        )
        , zi_2 as (
            select zi,min(cast(u_id as int)) as u_id
            from t_zi_part 
            where is_active='Y' and cast(u_id as int) > 0 and zi in (select zi from dup_zi)
            group by zi 
        )
        update t_zi_part 
            set is_active = '' 
        where 1=1
            and is_active='Y' 
            and zi in (select zi from zi_2)
            and cast(u_id as int) not in (select u_id from zi_2)
    """
    db_run_sql(sql_stmt, conn=_conn)

[DEBUG] 
        with dup_zi as (
            select zi,count(*) from t_zi_part where is_active='Y' and cast(u_id as int) > 0 group by zi having count(*) > 1
        )
        , zi_2 as (
            select zi,min(cast(u_id as int)) as u_id
            from t_zi_part 
            where is_active='Y' and cast(u_id as int) > 0 and zi in (select zi from dup_zi)
            group by zi 
        )
        update t_zi_part 
            set is_active = '' 
        where 1=1
            and is_active='Y' 
            and zi in (select zi from zi_2)
            and cast(u_id as int) not in (select u_id from zi_2)
    
